# Optimization of E. coli growth parameters for NUFEB
This notebook utilizes hyperparameter optimization to fit simulated E. coli growth curves to experimental data. Under the hood, we use scikit-optimize to perform a Bayesian optimization.

In [1]:
#Imports
import os
from random import uniform
from pathlib import Path
from nufeb_tools import utils,plot
import pandas as pd
from string import Template
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score,mean_squared_error
import seaborn as sns
from tqdm import tqdm
from joblib import Parallel
import joblib
import subprocess
from skopt import gp_minimize, forest_minimize, dummy_minimize, gbrt_minimize,load
from skopt.callbacks import CheckpointSaver
from skopt.plots import plot_convergence, plot_objective,plot_gaussian_process
from scipy.optimize import curve_fit
from scipy import interpolate
from skopt import Optimizer
from skopt.space import Real
from skopt.benchmarks import branin
from distributed import Client,progress, as_completed,wait,get_client
import shutil
from dask import dataframe as dd
from dask import delayed


In [2]:
client = Client(threads_per_worker=1)#

In [3]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 12
Total threads: 12,Total memory: 25.01 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42349,Workers: 12
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 25.01 GiB
Comm: tcp://127.0.0.1:43351,Total threads: 1
Dashboard: http://127.0.0.1:44925/status,Memory: 2.08 GiB
Nanny: tcp://127.0.0.1:42193,


In [4]:
expPath = r'/home/jonathan/nufeb-cyano-e-coli/experimental-data/ecw-growth-R2.xlsx'
growthData = pd.read_excel(expPath,sheet_name='Growth')
expSucrose =pd.read_excel(expPath,sheet_name='Sucrose')

In [5]:
growthData = growthData.loc[growthData['Initial Sucrose'] !=20]
growthData=growthData.groupby(['Time','Initial Sucrose']).mean().reset_index()[['Time','Initial Sucrose','OD600']]
growthData.head()

,Time,Initial Sucrose,OD600
0,0,2,0.010000
1,0,5,0.010000
2,0,10,0.010000
3,1,2,0.010000
4,1,5,0.016333


In [6]:
expSucrose = expSucrose.loc[expSucrose['Initial Sucrose'] !=20]
expSucrose = expSucrose.groupby(['Time','Initial Sucrose']).mean().reset_index()[['Time','Initial Sucrose','Sucrose']]
expSucrose.head()

,Time,Initial Sucrose,Sucrose
0,0,2,1.855295
1,0,5,4.685920
2,0,10,9.489184
3,4,2,1.808598
4,4,5,4.631757


In [7]:
expSucrose.loc[(expSucrose.Time ==0) & (expSucrose['Initial Sucrose']==2)].Sucrose

0    1.855295
Name: Sucrose, dtype: float64

# Optimization function
## Parameters to optimize:
### $\mu_{max}$ (Monod-based growth rate, $s^{-1}$)
### $\rho$ (Cell density, $\frac{kg}{m^3}$)
### $k_{sucrose}$ (Sucrose uptake half-maximum, $\frac{kg}{m^3}$)
### m (Cellular maintenance cost, $s^{-1}$)

In [8]:
#Volume = 1e-4*1e-4*1e-4 #m^3
#mlm3 = 1e6 #mL/m^3
#Biomass2OD = Volume*.44


In [9]:
def mu_func(x,K,r,N0):
    return K/(1  + ((K-N0)/N0)*np.exp(-r*x))

In [17]:
def run_sim(folder):
    os.chdir(folder)
    in_path = folder / 'Inputscript.lammps'
    try:
        os.system(f'mpirun -np 1 /home/jonathan/NUFEB/lammps/src/lmp_png -in {in_path} > nufeb.log')
        #subprocess.run(['mpirun', '-np', '4','/home/jsakkos/NUFEB/lammps/src/lmp_png','-in', '*.lammps > nufeb.log'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        #os.chdir(r'/home/jsakkos/NUFEB')
        return folder
    except:
        print('Something went wrong')
def collect_data(folder):
    SucroseMW = 342.3
    x = utils.get_data(directory=str(folder))
    Volume = np.prod(x.metadata['Dimensions'])
    Biomass2OD = .44*Volume*1e18
    temp = pd.concat([x.biomass.ecw.reset_index(drop=True)/Biomass2OD,x.biomass.step.reset_index(drop=True)/60/60*x.timestep,x.avg_con.Sucrose.reset_index(drop=True)],axis=1)
    temp.columns=['OD600','Hours','Sucrose']
    temp['Sucrose']=temp['Sucrose']*SucroseMW*1e-3
    temp['S0'] = temp.Sucrose.iloc[0]
    return temp
def run_collect(folder):
    os.chdir(folder)
    in_path = folder / 'Inputscript.lammps'
    os.system(f'mpirun -np 1 /home/jonathan/NUFEB/lammps/src/lmp_png -in {in_path} > nufeb.log')
    SucroseMW = 342.3
    x = utils.get_data(directory=str(folder))
    Volume = np.prod(x.metadata['Dimensions'])
    Biomass2OD = .44*Volume*1e18
    temp = pd.concat([x.biomass.ecw.reset_index(drop=True)/Biomass2OD,x.biomass.step.reset_index(drop=True)/60/60*x.timestep,x.avg_con.Sucrose.reset_index(drop=True)],axis=1)
    temp.columns=['OD600','Hours','Sucrose']
    temp['Sucrose']=temp['Sucrose']*SucroseMW*1e-3
    temp['S0'] = temp.Sucrose.iloc[0]
    return temp
def test(x,growth=growthData,sucrose=expSucrose):
   
    mu = x[0]
    rho = x[1]
    ksuc = x[2]
    #yield_ = x[3]
    maint = 0#x[3]
    SucroseMW = 342.3
    client = get_client()
    os.chdir('/home/jonathan/NUFEB')
    SucroseMW = 342.3
    rng = str(np.random.randint(0,1e6,1)[0])
    os.mkdir(rng)
    os.chdir(rng)
    dir_path = Path(os.getcwd()).resolve()
    for s in [10]: #growthData['Initial Sucrose'].unique()
        suc = expSucrose.loc[(expSucrose.Time ==0) & (expSucrose['Initial Sucrose']==s)].Sucrose.values[0]*1e3/SucroseMW
        text = f'nufeb-seed --n 1 --od 0,0.01 --d 1e-4,1e-4,1e-4 --grid 20 --t 8700 --suc {suc:.2f} --muecw {mu}  --rhoecw {rho} --ksuc {ksuc} --maintecw {maint} --yieldecw .46 --niter 1000 --mass 7e-12 --biodt 10'
        os.system(text)
    BASE_DIR = Path(f'runs/')
    folders = [path for path in BASE_DIR.resolve().iterdir() if path.is_dir()]
    futures =client.map(run_collect,folders)
    wait(futures,return_when='ALL_COMPLETED')
    data=futures.result()
    #data=pd.concat((futures[i].result() for i in range(3)),ignore_index=True) 
    df = data.groupby(['Hours','S0']).mean().reset_index()
    rmse = 0
    r2g=0
    r2s=0
    for s in [10]:
        temp = growthData.loc[growthData['Initial Sucrose']==s]
        sim_temp = df.loc[df.S0==s]
        g=interpolate.interp1d(sim_temp.Hours,sim_temp.OD600)
        rmse += mean_squared_error(temp.OD600, g(temp.Time), sample_weight=temp.OD600,squared = False)
        r2g += r2_score(temp.OD600, g(temp.Time))
        temp2 = expSucrose.loc[expSucrose['Initial Sucrose']==s]
        suc=interpolate.interp1d(sim_temp.Hours,sim_temp.Sucrose)
        rmse += mean_squared_error(temp2.Sucrose, suc(temp2.Time), sample_weight=temp2.Sucrose,squared = False)
        r2s += r2_score(temp2.Sucrose, suc(temp2.Time))
    r2g=r2g/len(growthData['Initial Sucrose'].unique())
    r2s=r2s/len(growthData['Initial Sucrose'].unique())

    print(f'R2 = {r2g:.3f} (growth),{r2s:.3f} (sucrose)')

    os.chdir('/home/jonathan/NUFEB')
    try:
        shutil.rmtree(dir_path)
    except OSError as e:
        print("Error: %s : %s" % (dir_path, e.strerror))
    return rmse


In [18]:
# os.chdir('/home/jonathan/NUFEB')
# os.system('nufeb-clean')
# SucroseMW = 342.3
# for s in growthData['Initial Sucrose'].unique():
#     suc = s*1e3/SucroseMW
#     text = f'nufeb-seed --n 1 --od 0,0.01 --d 1e-4,1e-4,1e-4 --grid 20 --t 8700 --suc {suc:.2f} --muecw 4e-4  --rhoecw 250 --ksuc .1 --maintecw 0 --niter 1000000 --mass 1e-11 --yieldecw 0.43 --biodt 10'
#     os.system(text)
# BASE_DIR = Path(f'runs/')
# folders = [path for path in BASE_DIR.resolve().iterdir() if path.is_dir()]
# futures =client.map(run_collect,folders)
# wait(futures,return_when='ALL_COMPLETED')
# data=pd.concat((futures[i].result() for i in range(3)),ignore_index=True) 
# #data=dd.concat((futures[i].result() for i in range(3)),ignore_index=True) 
# df = data.groupby(['Hours','S0']).mean().reset_index()
# f, axes = plt.subplots(ncols=2)
# sns.scatterplot(x='Time',y='OD600',hue='Initial Sucrose',data=growthData,ax=axes[0])
# sns.scatterplot(x='Time',y='Sucrose',hue='Initial Sucrose',data=expSucrose,ax=axes[1])
# sns.lineplot(x='Hours',y='OD600',hue='S0',data=df,ax=axes[0])
# sns.lineplot(x='Hours',y='Sucrose',hue='S0',data=df,ax=axes[1])
# sns.despine()
# plt.show()
# rmse = 0
# r2g=0
# r2s=0
# for s in growthData['Initial Sucrose'].unique():
#     temp = growthData.loc[growthData['Initial Sucrose']==s]
#     sim_temp = df.loc[df.S0==s]
#     g=interpolate.interp1d(sim_temp.Hours,sim_temp.OD600)
#     rmse += mean_squared_error(temp.OD600, g(temp.Time), sample_weight=temp.OD600,squared = False)
#     r2g += r2_score(temp.OD600, g(temp.Time))
#     temp2 = expSucrose.loc[expSucrose['Initial Sucrose']==s]
#     suc=interpolate.interp1d(sim_temp.Hours,sim_temp.Sucrose)
#     rmse += mean_squared_error(temp2.Sucrose, suc(temp2.Time), sample_weight=temp2.Sucrose,squared = False)
#     r2s += r2_score(temp2.Sucrose, suc(temp2.Time))
#     r2g=r2g/len(growthData['Initial Sucrose'].unique())
#     r2s=r2s/len(growthData['Initial Sucrose'].unique())

# print(f'R2 = {r2g:.3f} (growth),{r2s:.3f} (sucrose)')

In [19]:
# sns.scatterplot(x='Hours',y='OD600',data=df.loc[df.S0==10])
# sns.scatterplot(x='Time',y='OD600',data=growthData.loc[growthData['Initial Sucrose']==10])

In [20]:
# sns.scatterplot(x='Hours',y='Sucrose',data=df.loc[df.S0==10])
# sns.scatterplot(x='Time',y='Sucrose',data=expSucrose.loc[expSucrose['Initial Sucrose']==10])

In [21]:
# temp = growthData.loc[growthData['Initial Sucrose']==10]
# sim_temp = df.loc[df.S0==10]
# g=interpolate.interp1d(sim_temp.Hours,sim_temp.OD600)
# rmse += mean_squared_error(temp.OD600, g(temp.Time), sample_weight=temp.OD600,squared = False)
# r2g += r2_score(temp.OD600, g(temp.Time))
# temp2 = expSucrose.loc[expSucrose['Initial Sucrose']==10]
# suc=interpolate.interp1d(sim_temp.Hours,sim_temp.Sucrose)
# rmse += mean_squared_error(temp2.Sucrose, suc(temp2.Time), sample_weight=temp2.Sucrose,squared = False)
# r2s += r2_score(temp2.Sucrose, suc(temp2.Time))
# r2g=r2g/len(growthData['Initial Sucrose'].unique())
# r2s=r2s/len(growthData['Initial Sucrose'].unique())

# print(f'R2 = {r2g:.3f} (growth),{r2s:.3f} (sucrose)')

In [22]:
def check_result(x,growth=growthData,sucrose=expSucrose):
    mu = x[0]
    rho = x[1]
    ksuc = x[2]
    #yield_ = x[3]
    maint = 0#x[3]
    SucroseMW = 342.3
    client = get_client()
    os.chdir('/home/jonathan/NUFEB')
    SucroseMW = 342.3
    rng = str(np.random.randint(0,1e6,1)[0])
    os.mkdir(rng)
    os.chdir(rng)
    dir_path = Path(os.getcwd()).resolve()
    for s in [10]:
        suc = expSucrose.loc[(expSucrose.Time ==0) & (expSucrose['Initial Sucrose']==s)].Sucrose.values[0]*1e3/SucroseMW
        text = f'nufeb-seed --n 1 --od 0,0.01 --d 1e-4,1e-4,1e-4 --grid 20 --t 8700 --suc {suc:.2f} --muecw {mu}  --rhoecw {rho} --ksuc {ksuc} --maintecw {maint} --yieldecw .46 --niter 1000 --mass 7e-12 --biodt 10'
        os.system(text)
    BASE_DIR = Path(f'runs/')
    folders = [path for path in BASE_DIR.resolve().iterdir() if path.is_dir()]
    futures =client.map(run_collect,folders)
    wait(futures,return_when='ALL_COMPLETED')
    data = futures.result()
    #data=pd.concat((futures[i].result() for i in range(3)),ignore_index=True) 
    df = data.groupby(['Hours','S0']).mean().reset_index()
    f, axes = plt.subplots(ncols=2)
    sns.scatterplot(x='Time',y='OD600',hue='Initial Sucrose',data=growthData,ax=axes[0])
    sns.scatterplot(x='Time',y='Sucrose',hue='Initial Sucrose',data=expSucrose,ax=axes[1])
    sns.lineplot(x='Hours',y='OD600',hue='S0',data=df,ax=axes[0])
    sns.lineplot(x='Hours',y='Sucrose',hue='S0',data=df,ax=axes[1])
    #plot
    sns.despine()
    plt.show()
    rmse = 0
    r2g=0
    r2s=0
    for s in [10]:
        temp = growthData.loc[growthData['Initial Sucrose']==s]
        sim_temp = df.loc[df.S0==s]
        g=interpolate.interp1d(sim_temp.Hours,sim_temp.OD600)
        rmse += mean_squared_error(temp.OD600, g(temp.Time), sample_weight=temp.OD600,squared = False)
        r2g += r2_score(temp.OD600, g(temp.Time))
        temp2 = expSucrose.loc[expSucrose['Initial Sucrose']==s]
        suc=interpolate.interp1d(sim_temp.Hours,sim_temp.Sucrose)
        rmse += mean_squared_error(temp2.Sucrose, suc(temp2.Time), sample_weight=temp2.Sucrose,squared = False)
        r2s += r2_score(temp2.Sucrose, suc(temp2.Time))
        r2g=r2g/len(growthData['Initial Sucrose'].unique())
        r2s=r2s/len(growthData['Initial Sucrose'].unique())

    print(f'R2 = {r2g:.3f} (growth),{r2s:.3f} (sucrose)')
    os.chdir('/home/jonathan/NUFEB')
    try:
        shutil.rmtree(dir_path)
    except OSError as e:
        print("Error: %s : %s" % (dir_path, e.strerror))
    return rmse


In [23]:
optimizer = Optimizer(
    dimensions=[Real(float('5e-5'), float('8e-4')), 
                Real(150, 307),
                Real(float('1e-2'),float('1.5e1')),],
    random_state=1,
    base_estimator='gp'
)

futures = []
for x in optimizer.ask(n_points=10):
    x = [round(elem,8) for elem in x] 
    futures.append(client.submit(lambda x: (x, test(x)), x))
seq = as_completed(futures) # iterate over futures in completion order
for future in seq:
    x, y = future.result()
    optimizer.tell(x, y)
    if len(optimizer.Xi) > 200: # exit condition
        continue
    next_x = optimizer.ask()
    next_x = [round(elem,8) for elem in next_x] 
    seq.add(client.submit(lambda x: (x, test(x)), next_x))

print(min(optimizer.yi)) # print the best objective found

distributed.worker - WARNING - Compute Failed
Function:  lambda
args:      ([0.00014872, 273.87359886, 12.6747858])
kwargs:    {}
Exception: 'AttributeError("\'list\' object has no attribute \'result\'")'



AttributeError: 'list' object has no attribute 'result'

distributed.worker - WARNING - Compute Failed
Function:  lambda
args:      ([0.00067763, 280.01060291, 1.8317916])
kwargs:    {}
Exception: 'AttributeError("\'list\' object has no attribute \'result\'")'

distributed.worker - WARNING - Compute Failed
Function:  lambda
args:      ([0.00055707, 199.55586103, 14.47471084])
kwargs:    {}
Exception: 'AttributeError("\'list\' object has no attribute \'result\'")'

distributed.worker - WARNING - Compute Failed
Function:  lambda
args:      ([6.822e-05, 297.10493755, 12.44251631])
kwargs:    {}
Exception: 'AttributeError("\'list\' object has no attribute \'result\'")'

distributed.worker - WARNING - Compute Failed
Function:  lambda
args:      ([0.00015085, 253.11901707, 9.29097385])
kwargs:    {}
Exception: 'AttributeError("\'list\' object has no attribute \'result\'")'

distributed.worker - WARNING - Compute Failed
Function:  lambda
args:      ([0.00015923, 227.42854938, 13.33499182])
kwargs:    {}
Exception: 'AttributeError("\'list\' object h

In [ ]:
optimizer.get_result()

In [ ]:
optimizer.get_result().x

In [ ]:
plot_convergence(optimizer.get_result(),yscale='log')

In [ ]:
plot_objective(optimizer.get_result(),dimensions=[r'$\mu$',r'$\rho$',r'$K_{sucrose}$'],cmap='inferno')

In [ ]:
check_result(optimizer.get_result().x)